# Testing methods of Mishra et al. 2022

[link to paper](http://journals.sagepub.com/doi/10.1177/0272989X211044697)

I will use the Behavioral Risk Factor Surveillance System data set from the CDC. I have 5 datasets from 2011-2015 available on Kaggle.  I will probably start by dividing by race or gender unevenly in the train and recalibration sets and go from there.